In [12]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()


import pystan

## Model Legend

* m6 = AZ
* m11 = EZ
* m9 = EFA
* m10 = EFA.C

In [17]:
## 500 particles
# check on Mon 21.20
# 6 676
# 9 659
# 10 676
# 11 676
# 15 676


log_dirs = dict()
log_dirs['m6'] = 'log/fabian_batch/20210910_003532_big5_batch_m6_b3/'
log_dirs['m11'] = 'log/fabian_batch/20210910_003728_big5_batch_m11_b3/'
log_dirs['m10'] = 'log/fabian_batch/20210910_003713_big5_batch_m10/'
log_dirs['m9'] = 'log/fabian_batch/20210910_003631_big5_batch_m9/'
log_dirs['m15'] = 'log/fabian_batch/20210917_215455_big5_batch_m15_b3/'
    

model_nums = [6, 9, 10, 11, 15]
# model_nums = [6, 11]

for i in model_nums:
    print(i, load_obj('t', log_dirs['m'+str(i)]))

6 676
9 659
10 676
11 676
15 676


In [15]:
load_obj('t', log_dirs['m10'])

518

## Load Model Evidence

In [22]:
# log_dirs = dict()
# log_dirs['m6'] = 'log/fabian_real_muthen/20210829_201302_smallbig5_m6/'
# log_dirs['m11'] = 'log/fabian_real_muthen/20210829_201308_smallbig5_m11/'
# log_dirs['m10'] = 'log/fabian_real_muthen/20210829_201450_smallbig5_m10/'
# log_dirs['m9'] = 'log/fabian_real_muthen/20210829_201305_smallbig5_m9/'
# model_nums=[6, 9, 10, 11]

# load existing results
bf = dict()
for model in model_nums:
    bf['m'+str(model)] = np.cumsum(load_obj('log_lklhds', log_dirs['m'+str(model)])[30:])


In [26]:
bfdf = pd.DataFrame.from_dict(bf)
bfdf.rename({
    'm6':'AZ', 
    'm11':'EZ',
    'm9':'EFA',
    'm15':'SAT'
}, axis=1, inplace=True)

bfdf['t']  = np.arange(30, len(bfdf)+30)
for i in ['EZ', 'SAT']:
    bfdf['AZ / '+str(i)] = bfdf['AZ']-bfdf[str(i)]
bfdf.drop(['EFA', 'AZ', 'EZ', 'SAT'], axis=1, inplace=True)
bfdf = bfdf.melt(id_vars='t', var_name='Model')

In [27]:
width = 600
height = 400
c = alt.Chart(bfdf).mark_line(
    strokeWidth = 2,
    ).encode(
    alt.X('t:Q', title='Index i', scale=alt.Scale(domain=[0,677])),
    alt.Y('value:Q', title='Log Bayes Factor'),
    alt.Color('Model')
    ).properties(
    width=width, height=height
    ).configure_legend(
    titleFontSize=16,
    labelFontSize=14,
    symbolStrokeWidth=6
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).interactive()

c

alt.Chart(...)

In [6]:
np.log(100)

4.605170185988092

In [20]:
model_evidence = dict()
for m in ['M'+str(i) for i in model_nums]:
    model_evidence[m] = np.sum(bf[m][30:])

model_evidence

{'M6': -4133795.708684723, 'M11': -4180562.269038189}

## Scoring Rules


In [21]:
# load existing results
scrl = dict()
for model in model_nums:
    scrl['m'+str(model)] = np.cumsum(load_obj('scoring_rule', log_dirs['m'+str(model)])[31:])


In [22]:
scrldf = pd.DataFrame.from_dict(scrl)
scrldf.columns = ['CFA2',  'CFA1']
scrldf['t']  = np.arange(30, len(scrldf)+30)
for i in ['CFA1']:
    scrldf[str(i)+' - CFA2'] = scrldf[str(i)]-scrldf['CFA2']
scrldf.drop(['CFA2', 'CFA1'], axis=1, inplace=True)
scrldf = scrldf.melt(id_vars='t', var_name='Model')


# scrldf.columns = ['CFA2', 'EFA', 'EFA.C', 'CFA1']
# scrldf['t']  = np.arange(30, len(scrldf)+30)
# for i in ['EFA', 'EFA.C', 'CFA1']:
#     scrldf[str(i)+' - CFA2'] = scrldf[str(i)]-scrldf['CFA2']
# scrldf.drop(['CFA2', 'EFA', 'EFA.C', 'CFA1'], axis=1, inplace=True)
# scrldf = scrldf.melt(id_vars='t', var_name='Model')


In [23]:
width = 600
height = 400
c = alt.Chart(scrldf).mark_line(
    strokeWidth = 2,
    ).encode(
    alt.X('t:Q', title='Index i', scale=alt.Scale(domain=[0,676])),
    alt.Y('value:Q', title='Variogram Score'),
    alt.Color('Model')
    ).properties(
    width=width, height=height
    ).configure_legend(
    titleFontSize=16,
    labelFontSize=14,
    symbolStrokeWidth=6
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).interactive()

c


alt.Chart(...)